In [1]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
#from transformers import pipeline
# For fine-tuning
import torch
#from torch import nn
#from torch.utils.data import Dataset, DataLoader
# For processing
import numpy as np
import pandas as pd
import os
import json

/Users/taraksristy/courses/maddata/MadData2025/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folder_path = "./dreams"
count = 0
all_files = []
for i in os.listdir(folder_path):
    if i.endswith('.json'):
        count+=1
        all_files.append(i)
dreams = []
for i in all_files:
    full_name  = os.path.join(folder_path,i)
    with open(full_name,'r') as f:
        data = json.load(f)
        for x in data['dreams']:
            dreams.append({
                "content":x.get("content","")
            })
              
df = pd.DataFrame(dreams)
#print(count)
df.size

29345

In [3]:

#model from bert trained
model = BertModel.from_pretrained('bert-base-uncased')
#tokenizer will break the sentence down and split into word array and then 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "I had a dream about flying"
#this is the input that automozatically converts into nunmbers
inputs = tokenizer(
    text, 
    return_tensors='pt',
    max_length=512,        
    truncation=True,       
    padding=True           
)
#print(inputs)

#print("                                                                                         dinfosdnfosdnofsdn")
#another waybof going about it where we can break it down
inp = tokenizer.tokenize("I had a dream about flying")
#then convert each word into a numerical value
ids = tokenizer.convert_tokens_to_ids(inp)
#print(ids)
#convert the numerical array into a tensor
tens_ids = torch.tensor(ids)
#want to 
with torch.no_grad():
    outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state  
pooled_output = outputs.pooler_output  

#getting the themes use pca to reduce the vector to the most important
#themes

themes = []
symbols = []


In [7]:
#analysis
import tensorflow as tf

# a good k for th etop k elements with the greatest magnitude
# Assuming pooled_output is your BERT pooled output
tf_po = tf.constant(pooled_output.numpy())
#we need to normalize the vector to fidn the magnitude
mean = tf.reduce_mean(tf_po)

mag_vec = tf_po - mean
type(mean)
# 

tensorflow.python.framework.ops.EagerTensor

In [5]:
pooled_output

tensor([[-8.6610e-01, -2.3621e-01,  7.2882e-02,  5.1650e-01,  3.1173e-02,
         -1.5924e-01,  7.6193e-01,  2.0471e-01, -3.8466e-02, -9.9977e-01,
          4.0322e-02,  6.0115e-01,  9.8390e-01, -2.3796e-01,  9.3552e-01,
         -4.3444e-01, -7.1989e-02, -5.6387e-01,  1.9049e-01, -4.2264e-01,
          5.6773e-01,  9.6538e-01,  4.9300e-01,  2.6466e-01,  3.9429e-01,
          7.1327e-01, -5.2255e-01,  9.1905e-01,  9.4701e-01,  7.0310e-01,
         -5.7584e-01,  1.0717e-01, -9.8662e-01, -1.5214e-01,  2.2788e-01,
         -9.8486e-01,  1.4562e-01, -7.1173e-01,  9.7281e-02,  1.3138e-01,
         -9.1520e-01,  2.4200e-01,  9.9783e-01, -5.2602e-01, -1.9282e-02,
         -2.7214e-01, -9.9990e-01,  1.8164e-01, -8.5082e-01, -4.2768e-03,
          4.7878e-02, -8.6708e-02,  1.0950e-01,  3.3268e-01,  3.8085e-01,
          2.0378e-01, -2.1045e-01,  3.8861e-04, -1.8653e-01, -4.6691e-01,
         -6.1956e-01,  2.6956e-01, -1.4147e-01, -8.4301e-01,  2.0146e-01,
         -3.9060e-01,  1.7957e-02, -1.